## *Preparation des données*

### Importation des librairies 

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

### Importation des fichiers csv

In [2]:
aide_alimentaire = pd.read_csv("C:/Users/ocon_/Sync/1_Data_analyst/P4_ocon_jorge/Données/aide_alimentaire.csv")
dispo_alimentaire = pd.read_csv("C:/Users/ocon_/Sync/1_Data_analyst/P4_ocon_jorge/Données/dispo_alimentaire.csv")
population = pd.read_csv("C:/Users/ocon_/Sync/1_Data_analyst/P4_ocon_jorge/Données//population.csv")
sous_nutrition = pd.read_csv("C:/Users/ocon_/Sync/1_Data_analyst/P4_ocon_jorge/Données/sous_nutrition.csv")

In [33]:
print('numpy version is {}'.format(np.__version__))
print('pandas version is {}'.format(pd.__version__))
print('seaborn version is {}'.format(sns.__version__))

numpy version is 1.21.5
pandas version is 1.4.2
seaborn version is 0.11.2


### Copy des donées 
Pour n'est pas alteré les données d'origine

In [3]:
aide_alimentaire_prep = aide_alimentaire.copy()
dispo_alimentaire_prep = dispo_alimentaire.copy()
population_prep = population.copy() 
sous_nutrition_prep = sous_nutrition.copy()

### Correction des data types
D'un autre cote, le data type pour certains quantité n'etais pas correcte, par example la colonne qui exprime la quantite des personnes en sous nutrition etais classé comme etant du texte alors que il s'agissait d'une quantité.


In [4]:
sous_nutrition_prep["Valeur"] = pd.to_numeric(sous_nutrition_prep["Valeur"], errors='coerce')
population_prep["Année"] = population_prep["Année"].astype(str)
aide_alimentaire_prep["Année"] = aide_alimentaire_prep["Année"].astype(str)

### Homogeanisation des unitées 
Les unitées n'etaient pas egaux pour le meme type de donnée, il y a une population exprimé en millions et une autre exprimée en milles, une quantité de norriture exprimé en tonnes et une autre exprimé en milliers des tonnes. 

In [5]:
dispo_alimentaire_prep = dispo_alimentaire_prep * [1, 1, 1, 1000, 1000, 1, 1, 1, 1, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
sous_nutrition_prep["Valeur"] *= 1000

L'homogenisation des unitées inclus le changement de noms des colonnes pour les rendre plus clairs.

In [6]:
sous_nutrition_prep.rename(columns={"Valeur": "Personnes en sous nutrition en milles"}, inplace=True)
population_prep.rename(columns={"Valeur": "Population en milles"}, inplace=True)
aide_alimentaire_prep.rename(columns={"Valeur": "Tonnes"}, inplace=True)

## *1er mission : Marc et le Panorama de l’état de la malnutrition dans le monde*



### 4 points principaux

Informations pour l’année 2017 :

- la proportion de personnes en état de sous-nutrition ;

- le nombre théorique de personnes qui pourraient être nourries. Tu devrais pouvoir calculer ça à partir de la disponibilité alimentaire mondiale ;

- idem pour la disponibilité alimentaire des produits végétaux ;

- l’utilisation de la disponibilité intérieure, en particulier la part qui est attribuée à l’alimentation animale, celle qui est perdue et celle qui est concrètement utilisée pour l'alimentation humaine. Je crois que Julien avait trouvé un moyen de facilement calculer ces proportions



#### Tri pour avoir les données relatives à l'année 2017

In [7]:
sous_nutrition_prep = sous_nutrition_prep[sous_nutrition_prep['Année'] == '2016-2018'] 
population_prep = population_prep[population_prep['Année'] == '2017']

### 1er point : Creation de table qui montre la proportion des personnes en etat de sous nutrition en 2017

In [8]:
prop_sous_nutrition_2017 = pd.merge(sous_nutrition_prep[["Zone", "Personnes en sous nutrition en milles"]], population_prep[["Zone","Population en milles"]],on="Zone")
prop_sous_nutrition_2017['Pourcentage'] = prop_sous_nutrition_2017['Personnes en sous nutrition en milles']/prop_sous_nutrition_2017['Population en milles']*100
prop_sous_nutrition_2017 = prop_sous_nutrition_2017.sort_values(by=["Pourcentage"], inplace=False, ascending=False).round(2)

In [34]:
prop_sous_nutrition_2017.head(10)

,Zone,Personnes en sous nutrition en milles,Population en milles,Pourcentage
78,Haïti,5300.0,10982.37,48.26
157,République populaire démocratique de Corée,12000.0,25429.82,47.19
108,Madagascar,10500.0,25570.51,41.06
103,Libéria,1800.0,4702.23,38.28
100,Lesotho,800.0,2091.53,38.25
183,Tchad,5700.0,15016.75,37.96
161,Rwanda,4200.0,11980.96,35.06
121,Mozambique,9400.0,28649.02,32.81
186,Timor-Leste,400.0,1243.26,32.17
0,Afghanistan,10500.0,36296.11,28.93


### 2eme point : Nombre théorique de personnes qui pourraient être nourries

#### Besoin alimentaire par pays
L'information se trouve dans https://www.fao.org/ dans ***Données de la sécurité alimentaire***. 

Il etais important d'avoir des chiffres officiels qui prennent en compte les differences culturelles, morphologiques et geografiques de chaque pays.

#### Importation des données

In [10]:
besoin_alimentaires_2017 = pd.read_csv("C:/Users/ocon_/Sync/1_Data_analyst/P4_ocon_jorge/Données/besoin_energetiques_2017.csv")

#### Preparation des données des besoins alimentaires

Avant :

In [11]:
besoin_alimentaires_2017.head(4)

,Zone,Produit,Unité,Valeur
0,Afghanistan,Besoins énergétiques alimentaires minimaux (kc...,Kcal/personne/jour,1676
1,Afghanistan,Besoins énergétiques alimentaires moyens (kcal...,Kcal/personne/jour,2134
2,Afrique du Sud,Besoins énergétiques alimentaires minimaux (kc...,Kcal/personne/jour,1859
3,Afrique du Sud,Besoins énergétiques alimentaires moyens (kcal...,Kcal/personne/jour,2406


Après :

In [12]:
besoin_alimentaires_2017.drop(["Unité"], axis=1, inplace=True)
besoin_alimentaires_2017 = besoin_alimentaires_2017.set_index(["Zone", "Produit"], drop=True).unstack()["Valeur"].reset_index()
besoin_alimentaires_2017.head(4)

Produit,Zone,Besoins énergétiques alimentaires minimaux (kcal/personne/jour),Besoins énergétiques alimentaires moyens (kcal/personne/jour)
0,Afghanistan,1676,2134
1,Afrique du Sud,1859,2406
2,Albanie,1911,2490
3,Algérie,1781,2289


#### Filtrage et jointure des données necessaires pour faire cette analyse 
- Données relatives aux besoin alimentaire;
- données relatives à la population et;
- données relatives à la disponibilité alimentaire

In [13]:
dispo_alimentaire_prep_2 = dispo_alimentaire_prep[["Zone", "Origine", "Produit", "Disponibilité alimentaire (Kcal/personne/jour)"]].copy()
#dispo_alimentaire_prep = dispo_alimentaire_prep[dispo_alimentaire_prep["Produit"].str.contains("Boissons Alcooliques")==False]
dispo_alimentaire_prep_2 = dispo_alimentaire_prep_2[dispo_alimentaire_prep["Produit"] != "Boissons Alcooliques"]
dispo_alimentaire_prep_2 = dispo_alimentaire_prep_2.groupby(['Zone']).sum()
dispo_alimentaire_prep_2 = dispo_alimentaire_prep_2.reset_index()

In [14]:
nb_personnes_nourries = pd.merge(population_prep[["Zone", "Population en milles"]], dispo_alimentaire_prep_2[["Zone","Disponibilité alimentaire (Kcal/personne/jour)"]],
                                 on="Zone")
nb_personnes_nourries = pd.merge(nb_personnes_nourries[["Zone", "Population en milles","Disponibilité alimentaire (Kcal/personne/jour)"]],
                                              besoin_alimentaires_2017[["Zone", "Besoins énergétiques alimentaires minimaux (kcal/personne/jour)", "Besoins énergétiques alimentaires moyens (kcal/personne/jour)"]], on="Zone")

#### Analyse statistique 
Population * dipo alimenaire / besoin min = Nombre theorique des personnes qui pourraient etre alimentées avec une ingestion de norriture minimal
Population * dipo alimenaire / besoin moy = Nombre theorique des personnes qui pourraient etre alimentées avec une ingestion de norriture moyenne

In [15]:
nb_personnes_nourries["Disponibilité alimentaire par pays"] = nb_personnes_nourries["Population en milles"]*nb_personnes_nourries["Disponibilité alimentaire (Kcal/personne/jour)"]
nb_personnes_nourries["personnes nourries avec le min des calories par pays en milles"] = round(nb_personnes_nourries["Disponibilité alimentaire par pays"]/nb_personnes_nourries["Besoins énergétiques alimentaires minimaux (kcal/personne/jour)"],3)
nb_personnes_nourries["personnes nourries par pays en milles"] = round(nb_personnes_nourries["Disponibilité alimentaire par pays"]/nb_personnes_nourries["Besoins énergétiques alimentaires moyens (kcal/personne/jour)"],3)
nb_personnes_nourries.loc['Total']= nb_personnes_nourries.sum(numeric_only=True)

nb_personnes_nourries.head(3)

,Zone,Population en milles,Disponibilité alimentaire (Kcal/personne/jour),Besoins énergétiques alimentaires minimaux (kcal/personne/jour),Besoins énergétiques alimentaires moyens (kcal/personne/jour),Disponibilité alimentaire par pays,personnes nourries avec le min des calories par pays en milles,personnes nourries par pays en milles
0,Afghanistan,36296.113,2087.0,1676.0,2134.0,7.574999e+07,45196.890,35496.714
1,Afrique du Sud,57009.756,2968.0,1859.0,2406.0,1.692050e+08,91019.341,70326.249
2,Albanie,2884.169,3167.0,1911.0,2490.0,9.134163e+06,4779.782,3668.339


In [16]:
nb_personnes_nourries.tail()

,Zone,Population en milles,Disponibilité alimentaire (Kcal/personne/jour),Besoins énergétiques alimentaires minimaux (kcal/personne/jour),Besoins énergétiques alimentaires moyens (kcal/personne/jour),Disponibilité alimentaire par pays,personnes nourries avec le min des calories par pays en milles,personnes nourries par pays en milles
168,Viet Nam,94600.648,2723.0,1785.0,2299.0,2.575976e+08,1.443124e+05,112047.657
169,Yémen,27834.819,2217.0,1703.0,2174.0,6.170979e+07,3.623593e+04,28385.370
170,Zambie,16853.599,1924.0,1696.0,2159.0,3.242632e+07,1.911929e+04,15019.141
171,Zimbabwe,14236.595,2071.0,1734.0,2216.0,2.948399e+07,1.700345e+04,13305.049
Total,NaN,7291900.830,484454.0,314785.0,406521.0,2.074540e+10,1.131859e+07,8762488.212


### 3eme point : Nombre théorique de personnes qui pourraient être nourries avec des produits vegetaux

#### Filtrage et jointure des données necessaires pour faire cette analyse 

In [17]:
dispo_alimentaire_prep_vegetal = dispo_alimentaire_prep[["Zone", "Origine", "Produit", "Disponibilité alimentaire (Kcal/personne/jour)"]].copy()
#dispo_alimentaire_vegetal = dispo_alimentaire_tonnes_Kcal[dispo_alimentaire_tonnes_Kcal["Origine"].str.contains("animale")==False]
dispo_alimentaire_prep_vegetal = dispo_alimentaire_prep_vegetal[dispo_alimentaire_prep_vegetal["Origine"] != "animale"]
dispo_alimentaire_prep_vegetal = dispo_alimentaire_prep_vegetal[dispo_alimentaire_prep_vegetal["Produit"] != "Boissons Alcooliques"]
dispo_alimentaire_prep_vegetal = dispo_alimentaire_prep_vegetal.groupby(['Zone']).sum()
dispo_alimentaire_prep_vegetal = dispo_alimentaire_prep_vegetal.reset_index()

In [18]:
Table_nombre_des_personnes_nourries_VEG = pd.merge(population_prep[["Zone", "Population en milles"]], dispo_alimentaire_prep_vegetal[["Zone","Disponibilité alimentaire (Kcal/personne/jour)"]],on="Zone")
Table_nombre_des_personnes_nourries_VEG1 = pd.merge(Table_nombre_des_personnes_nourries_VEG[["Zone", "Population en milles","Disponibilité alimentaire (Kcal/personne/jour)"]],
                                              besoin_alimentaires_2017[["Zone", "Besoins énergétiques alimentaires minimaux (kcal/personne/jour)", "Besoins énergétiques alimentaires moyens (kcal/personne/jour)"]], on="Zone")

#### Analyse statistique 

Le meme analyse que pour le 2eme point

In [19]:
Table_nombre_des_personnes_nourries_VEG1["Disponibilité alimentaire par pays"] = Table_nombre_des_personnes_nourries_VEG1["Population en milles"]*Table_nombre_des_personnes_nourries_VEG1["Disponibilité alimentaire (Kcal/personne/jour)"]
Table_nombre_des_personnes_nourries_VEG1["Nombre theorique MIN des personnes nourries par pays"] = round(Table_nombre_des_personnes_nourries_VEG1["Disponibilité alimentaire par pays"]/Table_nombre_des_personnes_nourries_VEG1["Besoins énergétiques alimentaires minimaux (kcal/personne/jour)"],3)
Table_nombre_des_personnes_nourries_VEG1["Nombre theorique MOY des personnes nourries par pays"] = round(Table_nombre_des_personnes_nourries_VEG1["Disponibilité alimentaire par pays"]/Table_nombre_des_personnes_nourries_VEG1["Besoins énergétiques alimentaires moyens (kcal/personne/jour)"],3)
Table_nombre_des_personnes_nourries_VEG1.loc['Total']= Table_nombre_des_personnes_nourries_VEG1.sum(numeric_only=True)

In [20]:
Table_nombre_des_personnes_nourries_VEG1.head()

,Zone,Population en milles,Disponibilité alimentaire (Kcal/personne/jour),Besoins énergétiques alimentaires minimaux (kcal/personne/jour),Besoins énergétiques alimentaires moyens (kcal/personne/jour),Disponibilité alimentaire par pays,Nombre theorique MIN des personnes nourries par pays,Nombre theorique MOY des personnes nourries par pays
0,Afghanistan,36296.113,1871.0,1676.0,2134.0,6.791003e+07,40519.109,31822.881
1,Afrique du Sud,57009.756,2481.0,1859.0,2406.0,1.414412e+08,76084.564,58786.868
2,Albanie,2884.169,2182.0,1911.0,2490.0,6.293257e+06,3293.175,2527.412
3,Algérie,41389.189,2914.0,1781.0,2289.0,1.206081e+08,67719.313,52690.300
4,Allemagne,82658.409,2375.0,1948.0,2545.0,1.963137e+08,100777.064,77137.022


In [21]:
Table_nombre_des_personnes_nourries_VEG1.tail()

,Zone,Population en milles,Disponibilité alimentaire (Kcal/personne/jour),Besoins énergétiques alimentaires minimaux (kcal/personne/jour),Besoins énergétiques alimentaires moyens (kcal/personne/jour),Disponibilité alimentaire par pays,Nombre theorique MIN des personnes nourries par pays,Nombre theorique MOY des personnes nourries par pays
168,Viet Nam,94600.648,2148.0,1785.0,2299.0,2.032022e+08,113838.763,88387.208
169,Yémen,27834.819,2028.0,1703.0,2174.0,5.644901e+07,33146.807,25965.507
170,Zambie,16853.599,1818.0,1696.0,2159.0,3.063984e+07,18065.945,14191.683
171,Zimbabwe,14236.595,1893.0,1734.0,2216.0,2.694987e+07,15542.027,12161.496
Total,NaN,7291900.830,389613.0,314785.0,406521.0,1.708718e+10,9353202.959,7244902.056


### 4eme point : l’utilisation de la disponibilité intérieure

En particulier la part qui est attribuée à l’alimentation animale, celle qui est perdue et celle qui est concrètement utilisée pour l'alimentation humaine

In [22]:
dispo_alimentaire_prep.head()

,Zone,Produit,Origine,Aliments pour animaux,Autres Utilisations,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock
0,Afghanistan,Abats Comestible,animale,NaN,NaN,5.0,1.72,0.20,0.77,53000.0,NaN,NaN,53000.0,NaN,53000.0,NaN,NaN,NaN
1,Afghanistan,"Agrumes, Autres",vegetale,NaN,NaN,1.0,1.29,0.01,0.02,41000.0,2000.0,40000.0,39000.0,2000.0,3000.0,NaN,NaN,NaN
2,Afghanistan,Aliments pour enfants,vegetale,NaN,NaN,1.0,0.06,0.01,0.03,2000.0,NaN,2000.0,2000.0,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,Ananas,vegetale,NaN,NaN,0.0,0.00,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,Bananes,vegetale,NaN,NaN,4.0,2.70,0.02,0.05,82000.0,NaN,82000.0,82000.0,NaN,NaN,NaN,NaN,NaN


Tri des colonnes qu'on va utiliser et elimination des boisons alcoholiques comme source des calories

In [23]:
#dispo_alimentaire_prep_4 = dispo_alimentaire_prep.drop(["Disponibilité alimentaire (Kcal/personne/jour)", "Disponibilité alimentaire en quantité (kg/personne/an)", 
                                                           #"Disponibilité de matière grasse en quantité (g/personne/jour)", "Disponibilité de protéines en quantité (g/personne/jour)"], axis=1)
dispo_alimentaire_prep_4 = dispo_alimentaire_prep[dispo_alimentaire_prep["Produit"].str.contains("Boissons Alcooliques")==False].copy()
utilisation_dispo_alimentaire = dispo_alimentaire_prep[["Zone", "Disponibilité intérieure", "Aliments pour animaux",
                                                              "Nourriture", "Pertes"]]
utilisation_dispo_alimentaire = utilisation_dispo_alimentaire.groupby(["Zone"]).sum()
utilisation_dispo_alimentaire.head()

,Disponibilité intérieure,Aliments pour animaux,Nourriture,Pertes
Zone,,,,
Afghanistan,13515000.0,768000.0,10735000.0,1135000.0
Afrique du Sud,61256000.0,5309000.0,29812000.0,2193000.0
Albanie,4758000.0,660000.0,3476000.0,276000.0
Algérie,42630000.0,4352000.0,31729000.0,3753000.0
Allemagne,162275000.0,30209000.0,79238000.0,3781000.0


#### Analyse statistique

On ajoute le pourcentage des aliments qui ne sont pas consommées par les humaines par rapport a la disponibilité interieur du pays

In [37]:
utilisation_dispo_alimentaire["Aliments pour animaux %"] = round(utilisation_dispo_alimentaire["Aliments pour animaux"]/utilisation_dispo_alimentaire["Disponibilité intérieure"]*100,2)
utilisation_dispo_alimentaire["Norriture %"] = round(utilisation_dispo_alimentaire["Nourriture"]/utilisation_dispo_alimentaire["Disponibilité intérieure"]*100,2)
utilisation_dispo_alimentaire["Pertes %"] = round(utilisation_dispo_alimentaire["Pertes"]/utilisation_dispo_alimentaire["Disponibilité intérieure"]*100,2)
utilisation_dispo_alimentaire["Autre %"] = round(100 - utilisation_dispo_alimentaire["Pertes %"] - utilisation_dispo_alimentaire["Norriture %"] - utilisation_dispo_alimentaire["Aliments pour animaux %"],2)

utilisation_dispo_alimentaire.head(10) 

,Disponibilité intérieure,Aliments pour animaux,Nourriture,Pertes,Aliments pour animaux %,Norriture %,Pertes %,Autre %
Zone,,,,,,,,
Afghanistan,13515000.0,768000.0,10735000.0,1135000.0,5.68,79.43,8.40,6.49
Afrique du Sud,61256000.0,5309000.0,29812000.0,2193000.0,8.67,48.67,3.58,39.08
Albanie,4758000.0,660000.0,3476000.0,276000.0,13.87,73.06,5.80,7.27
Algérie,42630000.0,4352000.0,31729000.0,3753000.0,10.21,74.43,8.80,6.56
Allemagne,162275000.0,30209000.0,79238000.0,3781000.0,18.62,48.83,2.33,30.22
Angola,32029000.0,9259000.0,15059000.0,4799000.0,28.91,47.02,14.98,9.09
Antigua-et-Barbuda,79000.0,0.0,58000.0,0.0,0.00,73.42,0.00,26.58
Arabie saoudite,31274000.0,11841000.0,17956000.0,1040000.0,37.86,57.42,3.33,1.39
Argentine,114369000.0,7890000.0,32531000.0,3522000.0,6.90,28.44,3.08,61.58


In [36]:
pertes_monde = (utilisation_dispo_alimentaire['Nourriture'].sum()/utilisation_dispo_alimentaire['Pertes'].sum())
pertes_monde

10.74780580915058

## *2eme mission : Mélanie*

#### Trois interrogations
 - les pays pour lesquels la proportion de personnes sous-alimentées est la plus forte en 2017;
 - ceux qui ont le plus bénéficié d’aide depuis 2013;
 - ceux ayant le plus/le moins de disponibilité/habitant

#### 1er :  Les pays pour lesquels la proportion de personnes sous-alimentées est la plus forte en 2017;


In [25]:
prop_sous_nutrition_2017.head()

,Zone,Personnes en sous nutrition en milles,Population en milles,Pourcentage
78,Haïti,5300.0,10982.37,48.26
157,République populaire démocratique de Corée,12000.0,25429.82,47.19
108,Madagascar,10500.0,25570.51,41.06
103,Libéria,1800.0,4702.23,38.28
100,Lesotho,800.0,2091.53,38.25


#### 2eme : Les pays qui ont le plus bénéficié d'aide depuis 2013

In [26]:
les_plus_beneficies_aide_alimentaire = aide_alimentaire_prep.copy()
les_plus_beneficies_aide_alimentaire["Année"] = pd.to_numeric(les_plus_beneficies_aide_alimentaire["Année"], errors='coerce')

In [27]:
les_plus_beneficies_aide_alimentaire = les_plus_beneficies_aide_alimentaire.query("Année >= 2013").groupby("Pays bénéficiaire").sum().drop(["Année"], axis=1).reset_index()
les_plus_beneficies_aide_alimentaire.sort_values(by=["Tonnes"], inplace=True, ascending=False)
les_plus_beneficies_aide_alimentaire.head(5)

,Pays bénéficiaire,Tonnes
50,République arabe syrienne,1858943
75,Éthiopie,1381294
70,Yémen,1206484
61,Soudan du Sud,695248
60,Soudan,669784


#### 3eme : Les pays ayant le plus/le moins de disponibilité/habitant

In [28]:
dispo_alimentaire_prep_le_moins = dispo_alimentaire_prep[["Zone", "Disponibilité alimentaire (Kcal/personne/jour)"]]
dispo_alimentaire_prep_le_moins = dispo_alimentaire_prep_le_moins.groupby("Zone").sum()
dispo_alimentaire_prep_le_moins.sort_values(by=["Disponibilité alimentaire (Kcal/personne/jour)"], ascending=True)

,Disponibilité alimentaire (Kcal/personne/jour)
Zone,
République centrafricaine,1879.0
Zambie,1924.0
Madagascar,2056.0
Afghanistan,2087.0
Haïti,2089.0
...,...
Israël,3610.0
États-Unis d'Amérique,3682.0
Turquie,3708.0


#### Julien

In [29]:
cereales_2017 = pd.read_csv("C:/Users/ocon_/Sync/1_Data_analyst/P4_ocon_jorge/Données/cereales_total_2017.csv")

In [30]:
cereales_2017 = cereales_2017.set_index(["Zone", "?l?ment"], drop=True).unstack()["Valeur"].reset_index()
cereales_2017['Proportion aliments pour animaux %'] = round(cereales_2017['Aliments pour animaux']/cereales_2017['Disponibilit? int?rieure']*100,2)
cereales_2017['Proportion aliments pour norriture humaine %'] = round(cereales_2017['Nourriture']/cereales_2017['Disponibilit? int?rieure']*100,2)
cereales_2017 = cereales_2017.sort_values(by=["Proportion aliments pour animaux %"], inplace=False, ascending=False)
cereales_2017.head()

?l?ment,Zone,Aliments pour animaux,Disponibilit? int?rieure,Nourriture,Proportion aliments pour animaux %,Proportion aliments pour norriture humaine %
51,Danemark,7171,8591,697,83.47,8.11
79,Irlande,3253,4035,592,80.62,14.67
129,Pays-Bas,9909,13604,1626,72.84,11.95
55,Espagne,22618,31235,5227,72.41,16.73
44,Chypre,432,626,170,69.01,27.16


In [31]:
df = prop_sous_nutrition_2017.loc[prop_sous_nutrition_2017['Zone'] == 'france']
df.head()

,Zone,Personnes en sous nutrition en milles,Population en milles,Pourcentage
